# Este notebook lo usaré para hacer el modelado del TFM

Referencias:
* https://medium.com/dunder-data/selecting-subsets-of-data-in-pandas-6fcd0170be9c
* https://medium.com/@juan.7655/cleansing-messy-excel-files-with-pandas-95a2ba3d8b2d
* https://cmdlinetips.com/2018/02/how-to-subset-pandas-dataframe-based-on-values-of-a-column/
* https://datatofish.com/convert-pandas-dataframe-to-list/
* https://www.geeksforgeeks.org/python-pandas-dataframe-insert/
* https://thispointer.com/python-pandas-drop-columns-in-dataframe-by-label-names-or-by-index-positions/

In [1]:
import pandas as pd
import math
import timeit

path = "U:\MASTER INT. ARTIFICIAL\TFM_DATA\CLEANDATA\\" #Path para mi laptop en casa
#path = "F:\\DATA_TFM\\CLEANDATA\\" #Path para mi PC en el trabajo

archivo = "data_best_hits_inc_tselp.csv"
file_hits = path+archivo

archivo = "data_aafit_bbfit_muon.csv"
file_aafit = path+archivo

## Analizando un poco los datos ya procesados, completos y limpios (dataset).

<font size=3>Después de todo el trabajo de preprocesamiento y limpieza, tenemos que, estos están por el momento divididos en dos partes.

* Dataframe 1, el cual contiene los hits, mismo que proviene del archivo llamado **data_best_hits_inc_tselp.csv**
* Dataframe 2, el cual contiene la data de aafit, bbfit y el muon por frameID (evento), mismo que proviene del archivo **data_aafit_bbfit_muon.csv**
</font>

## DATAFRAME DE LOS HITS

In [2]:
#Veamos como es el archivo del dataset relativo a los hits
df_hits = pd.read_csv(file_hits)

#En cargar tarda "bastante" alrededor de 5 min en la PC del trabajo (Core i5, 32GB RAM, Disco SSD.), 
#en la laptop de casa, casi 8 min.

In [3]:
#Obtenemos información del dataframe
df_hits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63520301 entries, 0 to 63520300
Data columns (total 17 columns):
runID              int64
frameID            int64
trigger_counter    int64
interactionID      int64
#hit               int64
lin_det            int64
piso_det           int64
mód_opt            int64
X'                 float64
Y'                 float64
Z'                 float64
azimut             float64
zenit              float64
tiempo             float64
t_selp             float64
amp                float64
frec_hits          float64
dtypes: float64(9), int64(8)
memory usage: 8.0 GB


In [10]:
#Mostramos algunas líneas o las 10 primeras líneas para "ver" como son
df_hits.sample(20)

,runID,frameID,trigger_counter,interactionID,#hit,lin_det,piso_det,mód_opt,X',Y',Z',azimut,zenit,tiempo,t_selp,amp,frec_hits
35832125,61730,1738,46,1,250,6,12,0,92.536086,-21.949393,-10.892434,0.591511,2.356193,-500.388776,281.767246,1.209487,233.325958
37199447,62290,54398,3348,2,264,9,6,2,-94.831312,8.924228,-98.535434,2.721155,2.356193,-1029.412569,966.875462,0.537656,45.681000
42718636,66470,57177,73,1,40,6,24,2,92.421481,-22.719641,163.344566,-0.886765,2.356193,373.530305,329.272238,0.770934,81.710815
13227908,43040,31770,349,2,111,5,9,2,-58.555825,78.717306,-55.296892,1.874285,2.356193,408.222137,408.222137,1.171191,73.776245
2260468,29920,115413,420,2,140,4,12,0,44.457062,9.379997,-11.661892,2.647250,2.356193,351.606644,351.606644,0.963617,66.909790
22289074,52710,46570,1838,2,122,4,10,0,45.447996,9.856232,-40.735434,0.582420,2.356193,493.783154,493.783154,0.767983,68.912506
59569885,84010,191031,1710,1,88,9,19,0,-94.206152,8.118598,90.148566,-1.401195,2.356193,2158.954613,686.519816,0.745875,79.975128
54180791,79920,6482,1086,2,135,1,17,0,4.665254,97.196797,52.377566,1.006830,2.356193,919.794022,912.000729,1.648834,53.920746
25811306,54740,28479,1906,2,36,5,13,0,-58.956339,78.519677,2.812566,-3.005135,2.356193,192.358053,144.563336,0.963351,51.822662
23940824,53670,38010,4792,2,136,6,3,1,91.478998,-22.299222,-141.602434,-3.094001,2.356193,507.414116,375.433825,0.941140,56.781769


## DATAFRAME CON DATA DE AAFIT, BBFIT Y OTROS

In [5]:
#Contrastando con el dataframe obtenido para el punto 3.1 del email de Salva, abrimos el segundo dataframe
df_aafit = pd.read_csv(file_aafit)

In [6]:
#Obtenemos la info del segundo dataframe
df_aafit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2261213 entries, 0 to 2261212
Data columns (total 14 columns):
runID              int64
frameID            int64
trigger_counter    int64
interactionID      int64
aafit_azimut       float64
aafit_zenit        float64
aafit_lambda       float64
aafit_beta         float64
bbfit_azimut       float64
bbfit_zenit        float64
bbfit_quality      float64
muon_azimut        float64
muon_zenit         float64
muon_energy        float64
dtypes: float64(10), int64(4)
memory usage: 241.5 MB


In [11]:
df_aafit.sample(20)

,runID,frameID,trigger_counter,interactionID,aafit_azimut,aafit_zenit,aafit_lambda,aafit_beta,bbfit_azimut,bbfit_zenit,bbfit_quality,muon_azimut,muon_zenit,muon_energy
1097533,60130,39658,2752,2,1.770498,0.988097,-5.687637,0.007552,1.779879,0.988028,1.742711,2.194444,0.977883,85.42280
183725,33500,54576,436,2,-1.240496,0.700501,-5.866687,0.027226,-9.424778,0.617248,1.982299,-1.332754,0.763427,48.16520
428358,43210,133235,1517,1,2.520616,0.657097,-6.308875,0.018800,-9.424778,1.166725,1.611731,2.716027,1.217975,1685.06000
392094,41980,132057,187,1,-2.072738,0.922244,-6.003233,0.010258,-2.344349,0.847281,3.713074,-2.103128,0.829979,34.73040
1574394,69150,88420,1614,1,2.056294,1.954848,-6.136961,0.022206,1.443100,1.598474,6.500529,1.653828,2.319403,517.52100
1716891,74000,86670,643,1,0.364079,0.494131,-6.731321,0.025475,-9.424778,1.457988,2.224362,1.078800,1.895448,6.32631
1650089,71510,34270,874,1,1.485579,0.650762,-6.513786,0.019573,-9.424778,1.035725,3.524356,0.598241,0.859649,21.12240
1311719,63390,502,3483,2,-0.536384,2.855221,-5.402898,0.004291,-0.499332,2.855300,4.161811,-0.524741,2.858162,3347.10000
371638,41370,83829,512,2,2.129423,0.599631,-6.535160,0.034469,-9.424778,1.023501,1.510537,1.967919,0.477170,12.50360
1124660,60690,57431,3223,2,2.475524,0.925310,-6.133198,0.012213,-9.424778,1.218993,1.435849,-3.027757,1.085626,190.10400


## A partir de aquí todavía es trabajo en progreso

In [8]:
temp = df_aafit['runID'].unique()
temp
#En esta ultima corrida, ya aparecen desde el 25800 hasta el 85730, lo cual significa,
#que srecorrio todos los archivos.

1375


array([25800, 25880, 25920, ..., 85690, 85720, 85730], dtype=int64)

In [9]:
#Mostramos las 10 primeras líneas del segundo dataframe
#La idea es contrastar que tenemos respecto al runID 25800 en ambos dataframes
print(df_aafit[df_aafit['runID']==25800].shape)

df_hits[df_hits['runID']==25800].shape

(479, 14)


(18487, 17)